In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V


Python 3.12.4


In [3]:
import pickle
import pandas as pd

In [4]:
with open('homeworks/hw4/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

import requests
import pandas as pd
from io import BytesIO

def download_file(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we raise an exception for bad responses
    return BytesIO(response.content)

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    # Assume `categorical` is defined elsewhere in your script
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

# URL of the Parquet file
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet'

# Download the file and read it into a dataframe
file_content = download_file(url)
df = read_data(file_content)


In [6]:
# se demora 1.45 minutos
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
import numpy as np

std_y_pred = np.std(y_pred)
std_y_pred

np.float64(6.247488852238703)

In [8]:
year = 2023
month = 3


df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
def apply_model(df, X_val,  output_file):
    y_pred = model.predict(X_val)
    

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [16]:
apply_model(df, X_val, 'predictions.parquet')

In [17]:
import os

file_size = os.path.getsize('predictions.parquet')
print(f"The size of the predictions.parquet file is {file_size} bytes.")


The size of the predictions.parquet file is 68641880 bytes.
